In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
import flask

In [3]:
app = flask.Flask(__name__, template_folder='templates')

In [4]:
df1 = pd.read_csv("movie_dataset.csv")

all_titles = [df1['title'][i] for i in range(len(df1['title']))]

features = ['keywords','cast','genres','director']

def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

for feature in features:
    df1[feature] = df1[feature].fillna('') #filling all NaNs with blank string

df1["combined_features"] = df1.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column


In [5]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df1["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object
indices = pd.Series(df1.index, index=df1['title'])

cosine_sim = cosine_similarity(count_matrix)

df1 = df1.reset_index()

def get_recommendations(title):
    cosine_sim = cosine_similarity(count_matrix)
    idx = indices[title]
    similar_movies = list(enumerate(cosine_sim[idx])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it
    sim_scores = sorted(similar_movies,key=lambda x:x[1],reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    tit = df1['title'].iloc[movie_indices]
    dat = df1['overview'].iloc[movie_indices]
    dire = df1['director'].iloc[movie_indices]
    cas = df1['cast'].iloc[movie_indices]
    vote_avg = df1['vote_average'].iloc[movie_indices]
    genre = df1['genres'].iloc[movie_indices]
    link = df1['homepage'].iloc[movie_indices]
    return_df1 = pd.DataFrame(columns=['title','overview','director','cast','vote_average','genres','homepage'])
    return_df1['title'] = tit
    return_df1['overview'] = dat
    return_df1['director'] = dire
    return_df1['cast'] = cas
    return_df1['vote_average'] = vote_avg
    return_df1['genres'] = genre
    return_df1['homepage'] = link
    return return_df1

In [6]:
def trending():
    
    sim_scores=df1.groupby('title')['vote_average'].count().sort_values(ascending=False)
    sim_scores = indices[all_titles] 
    movie_indices = sim_scores[0:5]
    tit = df1['title'].iloc[movie_indices]
    genre = df1['genres'].iloc[movie_indices]
    
    return_df = pd.DataFrame(columns=['title','genres'])
    return_df['title'] = tit
    
    return_df['genres'] = genre
    
    return return_df

In [ ]:
# Set up the main route
@app.route('/', methods=['GET', 'POST'])

def main():
    if flask.request.method == 'GET':
        names = []
        links = []
        result_final = trending()
        for i in range(len(result_final)):
            names.append(result_final.iloc[i][0])
            links.append(result_final.iloc[i][1])
        return flask.render_template('index.html',movie_names=names,movie_links=links)
        
       
            
    if flask.request.method == 'POST':
        m_name = flask.request.form['movie_name']
        m_name = m_name.title()
        
        if m_name not in all_titles:
            return flask.render_template('negative.html',name=m_name)
        else:
            result_final = get_recommendations(m_name)
            names = []
            dates = []
            director = []
            cast = []
            rating = []
            genres = []
            links = []
            for i in range(len(result_final)):
                names.append(result_final.iloc[i][0])
                dates.append(result_final.iloc[i][1])
                director.append(result_final.iloc[i][2])
                cast.append(result_final.iloc[i][3])
                rating.append(result_final.iloc[i][4])
                genres.append(result_final.iloc[i][5])
                links.append(result_final.iloc[i][6])

            return flask.render_template('positive.html',movie_names=names,movie_date=dates,movie_director=director,movie_cast=cast,movie_rating=rating,movie_genres=genres,movie_links=links,search_name=m_name)

        
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jul/2020 14:28:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2020 14:28:22] "POST / HTTP/1.1" 200 -
